In [142]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

#1-1 데이터가져오기 - 셀레니움을 이용한 페이지 이동
url = 'https://minwon.police.go.kr/'
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(5)
driver.find_element_by_xpath('//*[@id="banner-info"]/div[3]/a').click()
time.sleep(2)
driver.find_element_by_id('tab3_1').click()
time.sleep(2)
driver.find_element_by_id('indexedA').click()

#1-2 원하는 데이터 찾고하나를 선택해서 원하는 정보 추출하기.
lis2 = driver.find_element_by_css_selector('div#pagination > div')
lis_click = lis2.find_elements_by_css_selector('*')
name=[]
place = []
#1-3 각 페이지별 반복문
for i in range(0,6):
    time.sleep(1)
    lis = driver.find_element_by_css_selector('#minwonTbody')
    soup = BeautifulSoup(lis.text, 'html.parser')
    for a in range(0,5):
        name.append(soup.getText().split('\n')[a].split()[2])
        place.append(' '.join(soup.getText().split('\n')[a].split()[3:]))
    lis2.find_elements_by_css_selector('*')[i+3].click()
#1-3 마지막 페이지 정보 추출
time.sleep(1)
lis = driver.find_element_by_css_selector('#minwonTbody')
soup = BeautifulSoup(lis.text, 'html.parser')
name.append(soup.getText().split('\n')[0].split()[2])
place.append(' '.join(soup.getText().split('\n')[0].split()[3:]))

In [143]:
#1-4 데이터프레임으로 만들고 저장하기
df = pd.DataFrame({'name':name, 'place':place})
df['place'][13] = '서울 은평구 진흥로 58'#서울서부
df['place'][20] = '서울 강동구 성내로 57'#서울강동
df.to_csv('data/problem.csv', index=False)

In [144]:
import requests
from urllib.parse import quote

#2-1 카카오 로컬 API사이트 접속 준비
with open('data/kakaoapikey.txt') as file:
    kakao_key = file.read()
kakao_key

위도=[]
경도=[]
df = pd.read_csv('data/problem.csv')
#2-1 카카오 로컬 API사이트 접속
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}
#2-2 반복문을 사용하여 모든 경찰서의 위도 경도 좌표구하기
for addr in df['place']:
    url = f'{local_url}?query={quote(addr)}'
    result = requests.get(url, headers=header).json()
    경도.append(float(result['documents'][0]['x']))
    위도.append(float(result['documents'][0]['y']))
#2-3 데이터프레임에 위도,경도 컬럼추가하기
df['위도']=위도
df['경도']=경도
df.head(5)

,name,place,위도,경도
0,서울중부경찰서,서울특별시 중구 수표로 27,37.563632,126.990021
1,서울종로경찰서,서울특별시 종로구 율곡로 46,37.575876,126.984551
2,서울남대문경찰서,서울특별시 중구 한강대로 410,37.554811,126.973549
3,서울서대문경찰서,서울특별시 서대문구 통일로 113,37.565085,126.967074
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16,37.571768,126.999273


In [145]:
# 3-1 서울 지도 나타내기
import folium
map = folium.Map(location=[37.559868,126.967109], zoom_start=12)
#3-2 툴팁에는 경찰서명, 팝업에는 주소를 표시
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.place[i], max_width=200),
        tooltip=df.name[i],
        #3-3 아이콘은 기본모양이 아닌것으로 바꾸고, 지도제목 변경
        icon = folium.Icon(color='red', icon='home')
    ).add_to(map)
title_html = '''
             <h3 align="center" style="font-size:20px"><b>서울소재 경찰서 위치</b></h3>
             '''
map.get_root().html.add_child(folium.Element(title_html))

map